
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

<i18n value="04aa5a94-e0d3-4bec-a9b5-a0590c33a257"/>

# モデルレジストリ (Model Registry)

MLflow Model Registryは、チームでMLモデルの共有、実験からオンラインテスト、本番までの共同作業、承認およびガバナンスワークフローとの統合、MLデプロイメントとそのパフォーマンスの監視を行うことができるコラボレーションハブです。 このレッスンでは、MLflow モデルレジストリを使用してモデルを管理する方法について説明します。

このデモノートブックではAirbnbのデータセットにscikit-learnを使用しますが、ラボではMLlibを使用します。

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) このレッスンで以下を行います。<br>
 - MLflowを使ったモデルの登録
 - モデルライフサイクルの管理
 - モデルのアーカイブと削除
 
<img src="https://files.training.databricks.com/images/icon_note_24.png"/> モデルサービングのエンドポイントを立ち上げたい場合は、<a href="https://docs.databricks.com/applications/mlflow/model-serving.html#requirements" target="_blank">クラスターの作成</a> 権限が必要です。

In [0]:
%run "./Includes/Classroom-Setup"

<i18n value="5802ff47-58b5-4789-973d-2fb855bf347a"/>

### モデルレジストリ (Model Registry)

MLflow Model Registry コンポーネントは、MLflow モデルの全ライフサイクルを共同管理するための集中型モデルストア、API のセット、および UI です。モデルのリネージ（どの MLflow Experiment と Run でモデルを作成したか）、モデルのバージョン管理、ステージ遷移（例：ステージングから本番へ）、アノテーション（例：コメントやタグ）、デプロイ管理（例：どの本番ジョブから特定のモデルバージョンをリクエストしたか）を提供します。

モデルレジストリは以下の機能を備えています：

* **中央リポジトリ:** MLflow Model RegistryにMLflowモデルを登録します。登録されたモデルは、一意の名前、バージョン、ステージ、その他のメタデータを持ちます。
* **モデルのバージョン管理：** 登録されたモデルの更新時にバージョンを自動的に追跡します。
* **モデルステージ：** モデルのライフサイクルを表現するために、「ステージング」や「プロダクション」のように、各モデルのバージョンにプリセットまたはカスタムのステージを割り当てます。
* **モデルステージの遷移：** モデルの新規登録または変更のイベントを記録します。ユーザー、変更内容、およびコメントなどの追加メタデータが自動的に記録されます。
* **CI/CDワークフローの統合:** コントロールとガバナンス強化のため、CI/CDパイプラインの一部として、モデルステージの遷移、リクエスト、レビューと承認変更を記録します。

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-4/model-registry.png" style="height:400px; margin:20px"/></div>

<img src="https://files.training.databricks.com/images/icon_note_24.png"/> モデルレジストリの詳細については <a href="https://mlflow.org/docs/latest/registry.html" target="_blank">MLflow docs</a> を参照してください。

<i18n value="7f34f7da-b5d2-42af-b24d-54e1730db95f"/>

### モデルの登録 (Registering a Model)

以下のワークフローは、UIでも純粋なPythonでも動作します。 このノートブックでは、純粋なPythonを使用します。

<img src="https://files.training.databricks.com/images/icon_note_24.png"/> 画面左側の「モデル」タブをクリックし、UIを探索してください。

<i18n value="cbc59424-e45b-4179-a586-8c14a66a61a1"/>

モデルを学習し、<a href="https://docs.databricks.com/applications/mlflow/databricks-autologging.html" target="_blank">autologging (オートロギング)</a> を使って、MLflowにログを記録します。オートロギングにより、明示的なログステートメントを使用せずに、メトリクス、パラメータ、モデルを記録することができます。

オートロギングを使うには、いくつかの方法があります。

1. 学習コードの前に **`mlflow.autolog()`** を呼び出します。これにより、インポートすると同時に、インストールした各サポートしているライブラリのオートロギングが有効になります。
1. 管理コンソールからワークスペース・レベルでオートロギングを有効にします
1. コードで使用する各ライブラリには、ライブラリ固有の autolog 呼び出しを使用します。(例: **`mlflow.spark.autolog()`** )

ここでは、数値特徴のみを使用して簡単なランダムフォレストを構築します。

In [0]:
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

df = pd.read_csv(f"{DA.paths.datasets}/airbnb/sf-listings/airbnb-cleaned-mlflow.csv".replace("dbfs:/", "/dbfs/"))
X_train, X_test, y_train, y_test = train_test_split(df.drop(["price"], axis=1), df[["price"]].values.ravel(), random_state=42)

with mlflow.start_run(run_name="LR Model") as run:
    mlflow.sklearn.autolog(log_input_examples=True, log_model_signatures=True, log_models=True)
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    signature = infer_signature(X_train, lr.predict(X_train))

<i18n value="1322cac5-9638-4cc9-b050-3545958f3936"/>

ワークスペースのほかのユーザーとぶつからないように、ユニークなモデル名を作成します。

モデル名は、空でないUTF-8文字列でなければならず、フォワードスラッシュ(/)、ピリオド(.)、コロン(:)を含むことができないことに注意してください。

In [0]:
model_name = f"{DA.cleaned_username}_sklearn_lr2"
model_name

<i18n value="0777e3f5-ba7c-41c4-a477-9f0a5a809664"/>

モデルを登録します。

In [0]:
run_id = run.info.run_id
model_uri = f"runs:/{run_id}/model"

model_details = mlflow.register_model(model_uri=model_uri, name=model_name)

<i18n value="22756858-ff7f-4392-826f-f401a81230c4"/>

**画面左の*モデル*タブを開いて、登録されているモデルを探索します。** 以下の点に注意してください。

* 誰がモデルを学習させたか、どのコードを使用したかを記録します。
* このモデルで行われたアクションの履歴を記録します。
* このモデルをバージョン１として記録します。

<div><img src="https://files.training.databricks.com/images/301/registered_model_new.png" style="height:600px; margin:20px"/></div>

<i18n value="481cba23-661f-4de7-a1d8-06b6be8c57d3"/>

状態を確認します。

In [0]:
from mlflow.tracking.client import MlflowClient

client = MlflowClient()
model_version_details = client.get_model_version(name=model_name, version=1)

model_version_details.status

<i18n value="10556266-2903-4afc-8af9-3213d244aa21"/>

次に、モデルの説明を追加します。

In [0]:
client.update_registered_model(
    name=model_details.name,
    description="This model forecasts Airbnb housing list prices based on various listing inputs."
)

<i18n value="5abeafb2-fd60-4b0d-bf52-79320c10d402"/>

バージョンに応じた説明を追加します。

In [0]:
client.update_model_version(
    name=model_details.name,
    version=model_details.version,
    description="This model version was built using OLS linear regression with sklearn."
)

<i18n value="aaac467f-3a52-4428-a119-8286cb0ac158"/>

### モデルのデプロイメント (Deploying a Model)

MLflow Model Registryは、4つのモデルステージを定義しています。 **`None`** ,  **`Staging`** , **`Production`** と **`Archived`** です。それぞれのステージには、固有の意味があります。例えば、 **`Staging`** はモデルのテスト用で、 **`Production`** はテストまたはレビュープロセスを完了し、アプリケーションにデプロイされたモデルのものです。

適切な権限を持つユーザーは、ステージ間でモデルを移行させることができます。

<i18n value="dff93671-f891-4779-9e41-a0960739516f"/>

ステージの遷移について学んだので、モデルを **`Production`** ステージに遷移させます。

In [0]:
import time

time.sleep(10) # In case the registration is still pending

In [0]:
client.transition_model_version_stage(
    name=model_details.name,
    version=model_details.version,
    stage="Production"
)

<i18n value="4dc7e8b7-da38-4ce1-a238-39cad74d97c5"/>

モデルの現在の状態を取得します。

In [0]:
model_version_details = client.get_model_version(
    name=model_details.name,
    version=model_details.version
)
print(f"The current model stage is: '{model_version_details.current_stage}'")

<i18n value="ba563293-bb74-4318-9618-a1dcf86ec7a3"/>

**`pyfunc`** を使って最新のモデルを取得します。 このようにモデルをロードすると、トレーニングに使用したパッケージに関係なく、モデルを使用することができます。

<img src="https://files.training.databricks.com/images/icon_note_24.png"/> 特定のバージョンのモデルも読み込むことができます。

In [0]:
import mlflow.pyfunc

model_version_uri = f"models:/{model_name}/1"

print(f"Loading registered model version from URI: '{model_version_uri}'")
model_version_1 = mlflow.pyfunc.load_model(model_version_uri)

<i18n value="e1bb8ae5-6cf3-42c2-aebd-bde925a9ef30"/>

モデルを適用します。

In [0]:
model_version_1.predict(X_test)

<i18n value="75a9c277-0115-4cef-b4aa-dd69a0a5d8a0"/>

### 新しいバージョンのモデルのデプロイ (Deploying a New Model Version)

MLflow Model Registryでは、登録された1つのモデルに対応する複数のモデルバージョンを作成することができます。ステージ遷移を行うことで、新しいバージョンのモデルをステージング環境またはプロダクション環境にシームレスに統合することができます。

<i18n value="2ef7acd0-422a-4449-ad27-3a26f217ab15"/>

新しいモデルのバージョンを作成し、ログに記録されたときにそのモデルを登録します。

In [0]:
from sklearn.linear_model import Ridge

with mlflow.start_run(run_name="LR Ridge Model") as run:
    alpha = .9
    ridge_regression = Ridge(alpha=alpha)
    ridge_regression.fit(X_train, y_train)

    # Specify the `registered_model_name` parameter of the `mlflow.sklearn.log_model()`
    # function to register the model with the MLflow Model Registry. This automatically
    # creates a new model version

    mlflow.sklearn.log_model(
        sk_model=ridge_regression,
        artifact_path="sklearn-ridge-model",
        registered_model_name=model_name,
    )

    mlflow.log_params(ridge_regression.get_params())
    mlflow.log_metric("mse", mean_squared_error(y_test, ridge_regression.predict(X_test)))

<i18n value="dc1dd6b4-9e9e-45be-93c4-5500a10191ed"/>

新モデルをステージングにします。

In [0]:
import time

time.sleep(10)

client.transition_model_version_stage(
    name=model_details.name,
    version=2,
    stage="Staging"
)

<i18n value="fe857eeb-6119-4927-ad79-77eaa7bffe3a"/>

新モデルのバージョンはUIで確認します。

<div><img src="https://files.training.databricks.com/images/301/model_version_new.png" style="height:600px; margin:20px"/></div>

<i18n value="6f568dd2-0413-4b78-baf6-23debb8a5118"/>

検索機能を使って、モデルの最新バージョンを取得できます。

In [0]:
model_version_infos = client.search_model_versions(f"name = '{model_name}'")
new_model_version = max([model_version_info.version for model_version_info in model_version_infos])

<i18n value="4fb5d7c9-b0c0-49d5-a313-ac95da7e0f91"/>

この新しいバージョンに説明を追加します。

In [0]:
client.update_model_version(
    name=model_name,
    version=new_model_version,
    description=f"This model version is a ridge regression model with an alpha value of {alpha} that was trained in scikit-learn."
)

<i18n value="10adff21-8116-4a01-a309-ce5a7d233fcf"/>

このモデルは現在ステージングにあるので、自動化CI/CDパイプラインを実行して、本番に入る前にモデルをテストすることができます。 それが完成すれば、そのモデルを本番に移行させることができます。

In [0]:
client.transition_model_version_stage(
    name=model_name,
    version=new_model_version,
    stage="Production", 
    archive_existing_versions=True # Archieve existing model in production 
)

<i18n value="e3caaf08-a721-425b-8765-050c757d1d2e"/>

バージョン1を削除します。 

<img src="https://files.training.databricks.com/images/icon_note_24.png"/> アーカイブされていないモデルを削除することはできません。

In [0]:
client.delete_model_version(
    name=model_name,
    version=1
)

<i18n value="a896f3e5-d83c-4328-821f-a67d60699f0e"/>

モデルのバージョン2もアーカイブします。

In [0]:
client.transition_model_version_stage(
    name=model_name,
    version=2,
    stage="Archived"
)

<i18n value="0eb4929d-648b-4ae6-bca3-aff8af50f15f"/>

ここで、登録されているモデル全体を削除します。

In [0]:
client.delete_registered_model(model_name)

<i18n value="6fe495ec-f481-4181-a006-bea55a6cef09"/>

## レビュー (Review)
* **質問：** MLflow のトラッキングは、モデルレジストリとはどのように違うのですか？ 
* **回答：** トラッキングは実験と開発のためのものです。 モデルレジストリは、モデルをトラッキングサーバからステージングを経てプロダクションに移行させるためのものです。 これは、データエンジニアや機械学習エンジニアがデプロイメントプロセスの中でよく担当する役割です。

* **質問：** なぜ、モデルレジストリが必要なのでしょうか？ 
* **回答：** MLflowのトラッキングが機械学習のトレーニングプロセスにエンドツーエンドの再現性を提供するように、モデルレジストリはデプロイメントプロセスの再現性とガバナンスを提供します。 本番システムはミッションクリティカルであるため、コンポーネントをACLで分離し、特定のユーザーのみが本番モデルを変更できるようにすることが可能です。 バージョン管理およびCI/CDワークフローの統合も、モデルを本番環境にデプロイする上で重要な側面です。

* **質問：** UIと比較して、プログラムでできることは何ですか？ 
* **回答：** ほとんどの操作は、UIを使用するか、純粋なPythonで行うことができます。 モデルのトラッキングはPythonで行う必要がありますが、そこから先はどちらでも可能です。 例えば、MLflow のトラッキング API を使って記録されたモデルは、UI を使って登録し、本番環境にプッシュすることができます。

<i18n value="ecf5132e-f80d-4374-a325-28b4e96d5b61"/>

## その他のトピックとリソース (Additional Topics & Resources)

**Q:** MLflow Model Registry の詳細資料は、どこにありますか？ 
**A:** <a href="https://mlflow.org/docs/latest/registry.html" target="_blank">MLflowドキュメンテーション</a> をご覧ください。

&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>